In [2]:
import pandas as pd 
from pandas.io.json import json_normalize
from ast import literal_eval
#1708345 1708337

In [15]:
MAXROWS = 1e4 # per CSV

i = rows = 100
for file in ['D:\GACRP\input/temp.csv',]:
    reader = pd.read_csv(file, usecols=['hits', 'fullVisitorId'], dtype={'fullVisitorId': 'str'}, chunksize = MAXROWS, skiprows=0)
    for chunk in reader:
        chunk.columns = ['fullVisitorId','hits']
        chunk['hits'][chunk['hits'] == "[]"] = "[{}]"
        chunk['hits'] = chunk['hits'].apply(literal_eval).str[0]
        column_as_df = json_normalize(chunk['hits'])
        column_as_df.columns = [f"hits.{subcolumn}" for subcolumn in column_as_df.columns]
        chunk = chunk.drop('hits', axis=1).merge(column_as_df, right_index=True, left_index=True)
        
        if 'product' in chunk.columns:
            #print(chunk['product'][0])
            chunk['v2ProductName'] = chunk['product'].apply(lambda x: [p['v2ProductName'] for p in x] if type(x) == list else [])
            chunk['v2ProductCategory'] = chunk['product'].apply(lambda x: [p['v2ProductCategory'] for p in x] if type(x) == list else [])
            del chunk['product']
        if 'promotion' in chunk.columns:
            #print(chunk['promotion'][0])
            chunk['promoId']  = chunk['promotion'].apply(lambda x: [p['promoId'] for p in x] if type(x) == list else [])
            chunk['promoName']  = chunk['promotion'].apply(lambda x: [p['promoName'] for p in x] if type(x) == list else [])
            del chunk['promotion']

        chunk.to_csv(f"hits-{i:03d}.csv", index=False)
        rows += len(chunk.index)
        print(f"hits-{i:05d}.csv written ({rows} cumulative rows)")
        i += 1



hits-00100.csv written (10100 cumulative rows)


In [4]:
train = pd.read_csv('D:\GACRP\input/train_v2.csv', nrows = 10000)

In [5]:
train.to_csv('temp.csv', index=False)